# MAG Sample: Get Partner Data

## Prerequisites

Complete these tasks before you begin this tutorial:

- Setting up provisioning of Microsoft Academic Graph to an Azure blob storage account. See [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning).
- Setting up Azure Databricks service. See [Set up Azure Databricks](https://docs.microsoft.com/academic-services/graph/get-started-setup-databricks).
- Install python library `plotly`, and `pycountry` on the cluster you want to run this tutorial.

## Gather the information

Before you begin, you should have these items of information:

- The name of your Azure Storage (AS) account containing MAG dataset from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The access key of your Azure Storage (AS) account from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The name of the container in your Azure Storage (AS) account containing MAG dataset.

## Import notebooks

- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) samples/pyspark/MagClass.py under your working folder.
- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) this notebook under the same folder.

### Initialize storage account and container details

  | Variable  | Value | Description  |
  | --------- | --------- | --------- |
  | AzureStorageAccount | Replace **`<AzureStorageAccount>`** | This is the Azure Storage account containing MAG dataset. |
  | AzureStorageAccessKey | Replace **`<AzureStorageAccessKey>`** | This is the Access Key of the Azure Storage account. |
  | MagContainer | Replace **`<MagContainer>`** | This is the container name in Azure Storage account containing MAG dataset, usually in the form of mag-yyyy-mm-dd. |
  | OutputContainer | Replace **`<OutputContainer>`** | This is the container name in Azure Storage account where the output goes to, this container needs to be created before running this script. |

In [0]:
AzureStorageAccount = '<AzureStorageAccount>'
AzureStorageAccessKey = '<AzureStorageAccessKey>'
MagContainer = '<MagContainer>'
OutputContainer = '<OutputContainer>'

### Define MicrosoftAcademicGraph class

Run the MagClass notebook to define MicrosoftAcademicGraph class.

In [0]:
%run "./MagClass"

### Create a MicrosoftAcademicGraph instance to access MAG dataset
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer.

In [0]:
mag = MicrosoftAcademicGraph(account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer)

### Create a AzureStorageUtil to access other Azure Storage files
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer.

In [0]:
asu = AzureStorageUtil(account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer)

### Load data previousely generated

In [0]:
# Get all paper details for the input organization from previous output
orgPapers = asu.load('Paper.tsv')

# Get all Paper-Author-Affiliation relationships for the input organization from previous output
orgPaperAuthorAffiliation = asu.load('PaperAuthorAffiliationRelationship.tsv')

# Get all paper-author-affiliation relationships
paperAuthorAffiliations = mag.getDataframe('PaperAuthorAffiliations')

# Get all affiliation details
affiliations = mag.getDataframe('Affiliations')

# Get all author details
authors = mag.getDataframe('Authors')

### Get partner Paper-Author-Affiliation relationships

In [0]:
# Get all Paper-Author-Affiliation relationships for papers published by the input organization
orgAllPaperAuthorAffiliations = paperAuthorAffiliations \
    .join(orgPapers, paperAuthorAffiliations.PaperId == orgPapers.PaperId, 'inner') \
    .select(orgPapers.PaperId, paperAuthorAffiliations.AuthorId, \
            paperAuthorAffiliations.AffiliationId, paperAuthorAffiliations.AuthorSequenceNumber)

# Get partner Paper-Author-Affiliation relationships by excluding those relationships of the input organization
partnerPaperAuthorAffiliation = orgAllPaperAuthorAffiliations.subtract(orgPaperAuthorAffiliation)
partnerPaperAuthorAffiliation.show(10)

+----------+----------+-------------+--------------------+
 PaperId| AuthorId|AffiliationId|AuthorSequenceNumber|
+----------+----------+-------------+--------------------+
2055627582|2048342568| 870545813| 3|
2055625840|2635939923| null| 1|
2055618074|3004182206| 150229711| 4|
2055625840|2037065463| null| 3|
2055627582|1968670148| 2800274787| 4|
2055627582|1983184564| 219193219| 2|
2055618074|2713374494| 150229711| 1|
2055618074|3022616436| 150229711| 2|
2055619136|2088944921| 887064364| 1|
2055619136|1073530958| null| 2|
+----------+----------+-------------+--------------------+
only showing top 10 rows

### Save PartnerPaperAuthorAffiliationRelationship.tsv

In [0]:
asu.save(partnerPaperAuthorAffiliation, 'PartnerPaperAuthorAffiliationRelationship.tsv')

### Get partner affiliations

In [0]:
# Get all partner affiliation Ids
partnerAffiliationIds = partnerPaperAuthorAffiliation \
    .where(partnerPaperAuthorAffiliation.AffiliationId.isNotNull()) \
    .select(partnerPaperAuthorAffiliation.AffiliationId) \
    .distinct()

# Get all partner affiliation details
partnerAffiliations = affiliations \
    .join(partnerAffiliationIds, affiliations.AffiliationId == partnerAffiliationIds.AffiliationId, 'inner') \
    .select(partnerAffiliationIds.AffiliationId, affiliations.DisplayName.alias('AffiliationName'))
partnerAffiliations.show(10)

+-------------+--------------------+
AffiliationId| AffiliationName|
+-------------+--------------------+
 177156846|South Dakota Stat...|
 45438204|University of Tex...|
 8408910|Cardiff Metropoli...|
 138211613|Austrian Academy ...|
 145608581| University of Miami|
 141988045|Saint Leo University|
 61057504|Fujian Agricultur...|
 57496824|National Institut...|
 169541294| Flinders University|
 1325736334|Battelle Memorial...|
+-------------+--------------------+
only showing top 10 rows

### Save PartnerAffiliation.tsv

In [0]:
asu.save(partnerAffiliations, 'PartnerAffiliation.tsv')

### Get partner authors

In [0]:
# Get all partner author Ids
partnerAuthorIds = partnerPaperAuthorAffiliation.select(partnerPaperAuthorAffiliation.AuthorId).distinct()
partnerAuthors = authors.join(partnerAuthorIds, partnerAuthorIds.AuthorId == authors.AuthorId) \
                        .select(partnerAuthorIds.AuthorId, authors.DisplayName.alias('AuthorName'))
partnerAuthors.show(10)

+---------+--------------------+
 AuthorId| AuthorName|
+---------+--------------------+
 1998877| Max Wintermark|
 16214300| Yassin Refahi|
 20502297| Iadine Chadès|
 36625720| David Bodoff|
166615085| Nico Görnitz|
212549371| Krina T. Zondervan|
221201208| Cosimo Laneve|
255829404| Henry Sowizral|
295111803| Adriel Dean-Hall|
319942754|Jason P. Leboe-Mc...|
+---------+--------------------+
only showing top 10 rows

### Save PartnerAuthor.tsv

In [0]:
asu.save(partnerAuthors, 'PartnerAuthor.tsv')